<header style="background-color: #00233C; color: #F0F0F0; padding: 20pt;">
    <h1 style="font-size: 36px; font-family: Arial; margin: 0;">
        Building Text-to-Teradata SQL Agents with LangChain and Amazon Bedrock
    </h1>
    <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
</header>

<section style="margin-top: 20px;">
    <p style="font-size: 18px; font-family: Arial;"><b>Introduction</b></p>
    <p style="font-size: 16px; font-family: Arial;">
        Data scientists, data engineers, and SQL experts are frequently tasked with querying enterprise databases to answer critical business questions, such as “What was our churn this month?” or “Which product led sales in the Americas?” The answers often trigger further follow-up questions, requiring engineers to spend significant time responding to data requests. 
        <br><br>
        By integrating Amazon Bedrock’s secure foundation models, LangChain’s open-source framework, and the powerful capabilities of Teradata Vantage™, we can develop text-to-SQL autonomous agents that enable business teams to independently retrieve insights. Leveraging teradataml, teams can analyze data stored in Vantage and external object stores, such as Amazon S3, Google Cloud, Azure Blob, and OTF, without needing to ingest it. This reduces data transfer costs and offers flexible exploration using text-to-SQL generative AI applications.
    </p>
</section>

<section style="margin-top: 20px;">
    <p style="font-size: 18px; font-family: Arial;"><b>Text-to-Teradata SQL Agent Architecture</b></p>
        <p style="font-size: 16px; font-family: Arial;">
            We will build a LangChain implementation of a text-to-SQL agent to generate and execute advanced SQL queries compatible with any LLM available via Amazon Bedrock. Users can ask business questions in natural English and receive answers with data drawn from the relevant databases. This agent will connect to a Vantage database to analyze data stored in Vantage and Amazon S3. You may add additional foreign tables to load data from Google Cloud, and Azure Blob.
        </p>
        <img src="LangChain-and-Amazon-Bedrock_Jupyter-NB_ImgCover.jpg" alt="Architecture for Text-2-TeradataSQL Agents with LangChain and Amazon Bedrock" style="width: 90%; height: auto; margin-top: 20px;">
    </div>
</section>

<section style="margin-top: 20px;">
    <ul style="font-size: 16px; font-family: Arial; list-style-type: none; padding: 0;">
        <li style="margin-bottom: 20px;">
            <b>Tools:</b>
            <div style="padding: 10px; margin-left: 40px; border-left: 4px solid #00BFA5;">
                <b>Amazon Bedrock</b> is a managed platform that allows developers to select, manage, and customize Foudation Models (FMs) with data from a single API. Amazon Bedrock ensures that all proprietary data fed to the selected LLM is managed and protected by AWS, with no data shared with model providers or used to improve base models. This provides a secure way for developing generative AI applications without compromising sensitive data. For this example, we will build our GenAI text-to-TeradataSQL with Anthropic Claude-3 offered via Bedrock.
            </div>
        </li>
        <li style="margin-bottom: 20px;">
            <div style="padding: 10px; margin-left: 40px; border-left: 4px solid #00BFA5;">
                <b>LangChain</b> provides an easy-to-use open-source framework with components, integaration and tools optimized for querying SQL databases, which speeds up the development process. When coupled with Amazon Bedrock's single API, which interfaces with a variety of chat models, LangChain accelerates the creation of POCs and the deployment of GenAI applications. These two tools together reduce the complexity and time involved in model selection and integration. With Bedrock, we can experiment with models from Anthropic, Cohere, Meta, Mistral AI, and Stability AI via one interface.
            </div>
        </li>
        <li>
            <div style="padding: 10px; margin-left: 40px; border-left: 4px solid #00BFA5;">
                <b>Teradata Vantage</b> facilitates the analysis of data stored externally in object storage. We can leverage the Teradataml library and its native object storage to read data inside Google Cloud, Azure Blob, Amazon S3, and OTFs (Open Table Formats). This minimizes costly data transfer and allows developers to work within Vantage using foreign tables, without needing to transfer data into Vantage or use additional storage in their Vantage environment.
            </div>
        </li>
    </ul>
</section>




<b style = 'font-size:20px;font-family:Arial'>Prerequisite</b>
- Must have access to AWS Console and Amazon Bedrock
- Ensure you have requested and been granted access to the <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">foundation model you want to use</a>. In this example, we are using Anthropic Claude 3 Sonnet. 

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the environment</b>


<p style="font-size: 16px; font-family: Arial;">
We begin by installing our dependencies. </p>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages

!pip install -r requirements.txt --quiet

In [ ]:
%%capture
!pip install pyOpenSSL cryptography --force-reinstall --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements will install the required libraries to run this demo. Be sure to restart the kernel after executing the above lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
    </div>

<hr style='height:1px;border:none;'>
<p style="font-size: 18px; font-family: Arial;"><b>1.1 Import the required libraries and set up environment</b></p>

<p style="font-size: 16px; font-family: Arial;">
    We will use the following libraries and tools:
</p>

<ul style="font-size: 16px; font-family: Arial; list-style-type: disc; padding-left: 20px;">
    <li>
        <b>boto3</b>: The AWS SDK for Python allows us to interact with AWS services, including the Amazon Bedrock API.
    </li>
    <li>
        <b>teradataml</b>: Enables enables us to establish a connection to our database using the <code>create_context()</code> function and allows us to create virtual DataFrames, which serve as references to database objects, allowing exploration of object storage data and enabling operations directly on Vantage without transferring entire datasets to the client, except when needed. For this demo, we will be exploring a dataset in S3 via a foreign table on Vantage.
    </li>
    <li>
        <b>LangChain’s SQLDatabase class </b>: A wrapper around the SQLAlchemy engine to facilitate interactions with databases using SQLAlchemy’s Python SQL toolkit and ORM capabilities.
    </li>
    <li>
        <b> LangChain’s create_sql_agent function</b>: A LangChain function to build a SQL agent by providing a language model and a database connection.
    </li>
    <li>
        <b>LangChain’s ChatBedrockConverse class</b>: A common interface for working with Amazons Bedrock's FM's that support chat functionalities.
    </li>
</ul>

In [ ]:
import os 
import getpass
import warnings
import boto3 

from langchain_community.utilities import SQLDatabase 
from langchain_community.agent_toolkits import create_sql_agent
from teradataml import *
 
from langchain_aws import ChatBedrockConverse
warnings.filterwarnings("ignore")

<hr style="height: 1px; border: none;">

<p style="font-size: 20px; font-family: Arial;"><b>2. Connect to Vantage</b></p>

<p style="font-size: 16px; font-family: Arial;">
    You will be prompted to enter the password. Once you enter the password, press the Enter key, and use the down arrow to move to the next cell.
    <br><br>
    Here, we use the <a href="https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/Context-to-Teradata-Vantage" target="_blank">create_context()</a> function to connect to the Vantage system using the teradatasql and teradatasqlalchemy DBAPI and dialect combination. This connection allows us to use the teradataml <a href="https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/DataFrames-for-Tables-and-Views/DataFrames-from-Teradata-Vantage-Data-Sources/DataFrame-Constructor" target="_blank">DataFrame()</a> constructor to build foreign tables for reading data from Object Storage - in this example from Amazon s3.
</p>


In [ ]:
password=getpass.getpass(prompt='Enter your ClearScape Analytics Environment password: ')

eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password, database = 'demo_user')
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO= AWS_Bedrock_LangChain_Text_to_SQL.ipynb;' UPDATE FOR SESSION;''')

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>2.1 Loading Data for This Demo</b></p>
<p style="font-size: 16px; font-family: Arial;">
    When working with Teradata Vantage for data analysis, you have two options:
</p>

<p style="font-size: 16px; font-family: Arial; margin-left: 20px;">
    <b>Analyze data stored externally in object storage.</b> This method uses native object storage integration to create <code>foreign tables</code> inside the database; point this virtual table to an external object storage location like Google Cloud, Azure Blob, and Amazon S3; and use SQL to analyze the data. The advantage here is that it minimizes data transfer and allows you to work within Vantage using foreign tables, without needing additional storage in your Vantage environment.
</p>

<p style="font-size: 16px; font-family: Arial; margin-left: 20px;">
    <b>Download data into your local Vantage environment.</b> Alternatively, you can use native object storage integration to ingest data at scale into Vantage using one SQL request. Downloading data can result in faster execution of some steps that perform the initial access to the source data.
</p>
<p style="font-size: 16px; font-family: Arial; margin-left: 20px;">
Let’s explore the data where it resides in Amazon S3 by creating a foreign table inside our <code> demo_user</code> with the following code:  
</p>

In [ ]:
load_data = """ CREATE FOREIGN TABLE demo_user.retail_marketing 
USING ( 
location('/s3/dev-rel-demos.s3.amazonaws.com/bedrock-demo/Retail_Marketing.csv') 
); """ 

try:
    execute_sql(load_data)
    print('Table Loaded')
except Exception as e:
    if "already exists" in str(e): 
        print("Table already exists, skipping creation.")
    else:
        print(f"An error occurred: {e}")

<hr style='height:2px;border:none;'>
<b style = 'font-size:28px;font-family:Arial'>3. Data Exploration</b>

<p style='font-size:16px;font-family:Arial'>We are working with data from a marketing campaign, which includes thousands of rows of customer data such as age, job, marital status, education, and more.</p>

<p style='font-size:16px;font-family:Arial'> We can categorize the input dataset into three main categories:</p>

<p style='font-size:16px;font-family:Arial'>
    <ol style='font-size:16px;font-family:Arial'>
        <li>Customer data, including age, profession, education, monthly income, and more.</li>
        <li>Attributes related to the last contact of the current campaign, such as contact, month, day, and so on.</li>
        <li>Other attributes, including campaign, previous outcome, payment methods, and more.</li>
        <li>The target attribute - whether the customer purchased the product.</li>
    </ol>
</p>
<p style='font-size:16px;font-family:Arial'>We have loaded the source data from <a href="https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset">Kaggle</a> into S3 and supplemented it with additional information such as city, monthly income, family members, and more. The data is then loaded into a Vantage foreign table named <i>Retail_Marketing</i>.</p>


<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>3.1 Examine the Retail Marketing Campaign table</b></p>    
<p style = 'font-size:16px;font-family:Arial'>Let's look at the data in the Retail_Marketing foreign table. This data resides in Object Storage, Amazon S3, and has not been ingested into our Vantage enviroment. </p>

In [ ]:
df = DataFrame(in_schema('demo_user', 'retail_marketing'))
df.shape

In [ ]:
df

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4. Create engine using SQLAlchemy </b><p>
<p style="font-size: 16px; font-family: Arial;">
 We create our engine using LangChain's SQLAlchemy wrapper <a href="https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.sql_database.SQLDatabase.html" target="_blank">SQLDatabase</a>. <br><br>The <code>db.get_usable_table_names()</code> method retrieves and prints the names of tables in the connected database. For this demonstration, we're using our default <code>demo_user</code> database. If there are no tables in your database, this method will return an empty array.
</p>
</p>


In [ ]:
db = SQLDatabase(eng)

print(db.dialect)
print(db.get_usable_table_names())

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>4. Set up the Bedrock client</b>
<p style="font-size: 16px; font-family: Arial;">
    Next, initialize the <code>bedrock-runtime</code> client. Because we are executing this demo outside of SageMaker we will pass in or <a/ href="https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html"> Amazon credentials<a>

In [ ]:
# If executing from sagemaker you do not need to define aws_access_key_id, aws_secret_access_key, or aws_session_token. Simply define client as 'bedrock-runtime'.
# boto3_bedrock = boto3.client('bedrock-runtime')

boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name=getpass.getpass(prompt='Enter your AWS region: '),
    aws_access_key_id=getpass.getpass(prompt='Enter your AWS Access Key ID: '),
    aws_secret_access_key=getpass.getpass(prompt='Enter your AWS Secret Access Key: '),
    aws_session_token=getpass.getpass(prompt='Enter your AWS Session Token: ')
)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Define LLM</b></p>
<p style="font-size: 16px; font-family: Arial;">
     First ensure you have requested and been granted access to the <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">foundation model you want to use</a>. In this example, we are using Anthropic Claude 3 Sonnet. 
</p>   
<img src="bedrock-claude-3.png" alt="Bedrock Claude 3" style="width: 90%; height: auto;">
<p style="font-size: 16px; font-family: Arial;">
    Define the LLM using the <code>ChatBedrockConverse</code> interface. When defining <code>ChatBedrockConverse</code>, set the <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns">Amazon Bedrock base model ID</a>, the client as <code>boto3_bedrock</code>, and the common inference parameters.
</p> 
<p style="font-size: 16px; font-family: Arial;">
    We use the optional parameter <b>temperature</b> to make our Teradata SQL outputs more predictable.
</p>

<div style="margin-left: 16px; font-size: 16px; font-family: Arial;">
    <b>- Temperature:</b> which can range from 0.0 to 2 and controls how creative our results will be, Setting it to 0.1 ensures the model favors higher-probability (more predictable) words, resulting in more consistent and less varied outputs.<br>
</div>

<p style="font-size: 16px; font-family: Arial;">
    For a complete list of optional parameters for base models provided by Amazon Bedrock, visit the <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html"> AWS docs</a>.
</p>

In [ ]:
llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.1,
    client=boto3_bedrock,
    max_tokens=None
    # other params...
)

<hr style='height:2px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.2 Create and Invoke the SQL agent</b></p>    


<p style="font-size: 16px; font-family: Arial;">
    With the connection to Teradata Vantage established and our database (<code>db</code>) and Large Language Model (<code>LLM</code>) defined, we are ready to create and invoke our SQL Agent using the <code>create_sql_agent()</code> function. 
    </p>
<p style="font-size: 16px; font-family: Arial;">
    We pass in our <code>llm</code> and <code>db</code> as required parameters and set <code>agent_type</code> to "zero-shot-react-description" to instruct the agent to perform a reasoning step before acting.  
    </p>
<p style="font-size: 16px; font-family: Arial;">
    We set <code>verbose</code> to true so that the agent can output detailed information of intermediate steps. Additionally, we set <code>handle_parsing_errors</code> to <code>True</code>, ensuring that errors are sent back to the LLM as observations, for the LLM to attempt handling the errors.
    </p>


In [ ]:
agent=create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>5. Set optional observability with LangSmith</b>
<p style="font-size: 16px; font-family: Arial;">
    We add optional additional observability with LangSmith. You can create a <a href="https://docs.smith.langchain.com/#2-create-an-api-key"> LangSmith free account with limited traces here.</a>
</p>

<p style="font-size: 16px; font-family: Arial;">
     To enhance observability in our application, we can configure environment variables for LangSmith. LangSmith comes pre-installed with the LangChain library, so all you need to do is generate a LangSmith API key and set up the following environment variables to start monitoring your applications. Uncomment the LangSmith variables if you want to use the LangSmith tracing functionalities to log and view executions of your LLM application. 
</p>

In [ ]:
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass() 
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "text-to-teradata-sql"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>6. Invoke the SQL Agent to explore the data</b>

<p style="font-size: 16px; font-family: Arial;">
    Invoke the agent with an exploratory command or question and observe each step. For example, you can ask it to describe the retail marketing table.
</p>

<p style="font-size: 16px; font-family: Arial;">
    As soon as we invoke the agent, it begins performing a sequence of thought and action. Notice how this behavior is very similar to how a Data Scientist would approach interacting with a database. 
<p style="font-size: 16px; font-family: Arial;">
Data scientists and data analysts usually start with a trial query to understand the table schema and look at initial rows. This exploration helps in constructing queries. If errors happen, they edit their queries until they are successful. Similarly, LangChain agents ensure LLMs are firmly rooted in real data by describing the database, including its table structure, data samples of the top few columns, and sample SELECT queries. This reduces hallucinations and ensures more reliable and accurate SQL query generation.  
</p>

In [ ]:
try:
    question = "Describe the retail marketing table"
    response = agent.invoke(question)
    if isinstance(response, dict) and 'output' in response:
        answer = response['output']
    else:
        answer = "No valid response received."
    print(f"Query: {question}\nResponse: {answer}")
except Exception as e:
    print(f"An error occurred: {e}")

<p style="font-size: 16px; font-family: Arial;">
If we inspect our application using LangSmith we can see all inputs and outputs in every step of the chain. You can view a public run of this example question
 <a href="https://smith.langchain.com/public/cc66a546-acf3-4e51-8044-cc1c194ed386/r">here</a>.  Note that the first call to Anthropic Claude 3 is the third step in our chain. In this third step, the input includes additional instructions for the agent to use supplementary tools such as sql_db_query, sql_db_list_tables, sql_db_query_checker, and sql_db_query, along with a specified format for thought and action sequences. This SQLDatabase agent has been engineered with prompts to emulate the approach taken by data scientists when exploring and analyzing data. </p>


<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>7. Optimizing our SQL Agent with Prompt Engineering</b>


<p style = 'font-size:18px;font-family:Arial'><b>7.1 Invoking the SQL Agent with a complex query</b></p>    


<p style="font-size: 16px; font-family: Arial;">
   Now let's ask our agent a slightly more involved question. You may notice that the queries generated by the LLM sometimes return syntax errors. However, the LLM might be able to recover from these errors and still provide the correct answer. In some cases, though, the LLM may arrive to the answer and the agent is stumped by a parsing error.
 
</p>

In [ ]:
try:
    question = "What is the month with the highest number of marketing engagements?"
    response = agent.invoke(question)
    if isinstance(response, dict) and 'output' in response:
        answer = response['output']
    else:
        answer = "No valid response received."
    print(f"Query: {question}\nResponse: {answer}")
except Exception as e:
    print(f"An error occurred: {e}")

<p style="font-size: 16px; font-family: Arial;">
The agent can answer various questions about our data using only table names, schemas, sample rows, and the LLM’s knowledge of Teradata SQL. Although it can recover from mistakes as noted in the previous query, there's significant room for improvement. The last query resulted in our agent using 23,913 tokens and taking 49.56 seconds to provide the correct answer. You can view a public view of this execution via this <a href="https://smith.langchain.com/public/878e5908-abaa-49fc-8157-781a44adc0a5/r" >LangSmith trace </a>. </p>
<hr style='height:2px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>7.2 Adding a custom prompt to the SQL Agent </b></p>    

<p style="font-size: 16px; font-family: Arial;">
 We can optimize the agents performance with additional prompt engineering. 
</p>

<p style="font-size: 16px; font-family: Arial;">
We import a <code>ChatPromptTemplate</code> class to build flexible reusable prompts in our agent. Here we define a prefix, format instructions, and a suffix and join them to create a custom prompt. The prefix has unique rules that apply to Teradata. The format guides it's Question, thought, observation behavior and the suffix cues it to begin. 
</p>


In [ ]:
from langchain_core.prompts import ChatPromptTemplate    

prefix = """You are an helpful and expert TeradataSQL database admin. TeradataSQL shares many similarities to SQL, with a few key differences.
Given an input question, first create a syntactically correct TeradataSQL query to run, then look at the results of the query and return the answer.

IMPORTANT: Unless the user specifies an exact number of rows they wish to obtain, you must always limit your query to at most {top_k} results by using "SELECT TOP {top_k}".

The following keywords do not exist in TeradataSQL: 
1. LIMIT 
2. FETCH
3. FIRST
Instead of LIMIT or FETCH, use the TOP keyword. The TOP keyword should immediately follow a "SELECT" statement.
For example, to select the top 3 results, use "SELECT TOP 3 FROM <table_name>"
Enclose all value identifiers in quotes to prevent errors from restricted keywords. Append an underscore to all alias keywords (e.g., AS count_).
Always use double quotation marks (" ") for column names in SQL queries to avoid syntax errors.
NOT make any DML statements (INSERT, UPDATE, DELETE, DROP, etc.) to the database. 
If the question does not seem related to the database, just return "I don't know" as the answer

You have access to the following tools:"""

format_instructions = """You must always the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Don't forget to prefix your final answer with the string, "Final Answer:"!"""

suffix = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""

custom_prompt = ChatPromptTemplate.from_template("\n\n".join([
    prefix,
    "{tools}",
    format_instructions,
    suffix,
]))


agent=create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True,
    prompt=custom_prompt
)

<hr style='height:2px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>7.3 Test and Compare Results </b></p>    


<p style="font-size: 16px; font-family: Arial;">
To test and compare our results let's invoke the agent using the same question as the previous step. </p>

In [ ]:
try:
    question = "What is the month with the highest number of marketing engagements?"
    response = agent.invoke(question)
    if isinstance(response, dict) and 'output' in response:
        answer = response['output']
    else:
        answer = "No valid response received."
    print(f"Query: {question}\nResponse: {answer}")
except Exception as e:
    print(f"An error occurred: {e}")

<p style="font-size: 16px; font-family: Arial;">
This time our response was returned within 19.30 seconds and required 5,692 tokens! Inspect this public run on <a href="https://smith.langchain.com/public/e5cff28b-9bfe-48ba-a990-b87b0c3e2e06/r">LangSmith</a>. This time the LLM did not generate the query using the restricted `LIMIT` keyword. This additional prompting enables it to produce the final answer with fewer iterations. 
Final answer: The month with the highest number of marketing engagements is May. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>7.4 You can try your own question</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here are some sample questions that you can try out:</p>

<ol style = 'font-size:16px;font-family:Arial'>
    <li>What is the average income for Phoenix?</li>
    <li>Which city has the highest average income?</li>
    <li>What is the average age of married people?</li>
    <li>Which profession has the most married people in Phoenix?</li>
    <li>What is the month with the lowest sales?</li>
    <li>What is the month with the highest number of marketing engagements?</li>
    <li>What is the payment method distribution?</li>
    <li>What is the average number of days between a customer's last contact and their next purchase?</li>
    <li>What is the relationship between marital status and purchase frequency?</li>
    <li>What is the most effective communication method for reaching customers who have not purchased from our company in the past 6 months?</li>
</ol>

In [ ]:
try:
    question = input("\nEnter your natural language query: ")
    response = agent.invoke(question)
    if isinstance(response, dict) and 'output' in response:
        answer = response['output']
    else:
        answer = "No valid response received."
    print(f"Query: {question}\nResponse: {answer}")
except Exception as e:
    print(f"An error occurred: {e}")


<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>8. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
db_drop_table(table_name='retail_marketing') 

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>